In [12]:
import pandas as pd

ene =6.0
ene_str = str(ene).replace('.', '-')

# Initialize lists to store DataFrames
dfs_bad = []
dfs_good = []

# Loop over the CSV files and append to the lists
for i in (1,2):
    dfs_bad.append(pd.read_csv(f'df{ene_str}_bad{i}.csv'))
    dfs_good.append(pd.read_csv(f'df{ene_str}_good{i}.csv'))

# Concatenate all dataframes in the list into a single dataframe
df_bad = pd.concat(dfs_bad, ignore_index=True)
df_good = pd.concat(dfs_good, ignore_index=True)

FileNotFoundError: [Errno 2] No such file or directory: 'df6-0_bad1.csv'

In [ ]:
df_good

,beta_hat,eventNumber
0,0.984275,637
1,0.984275,637
2,0.984275,637
3,0.984275,637
4,0.984275,637
...,...,...
72783107,0.984553,99189
72783108,0.984553,99189
72783109,0.984553,99189
72783110,0.984553,99189


In [ ]:
from scipy.optimize import fsolve

# Define the function
def calculate_half_angle(R, theta0=0.7):    
    def func(theta, R1=R, n=1.05, h=90, L=765):
        return L * np.tan(np.arcsin(n * np.sin(theta))) + (h / 2) * np.tan(theta) - R1

    theta_sol = fsolve(func, theta0, xtol=1e-2)

    return theta_sol[0]

def calculate_beta(cherenkov_angle, refractive_index):
    # Check if refractive index is non-zero to avoid division by zero
    if np.any(refractive_index == 0):
        raise ValueError("Refractive index cannot be zero.")

    # Calculate beta using the Cherenkov angle and refractive index
    beta = 1 / np.cos(cherenkov_angle) / refractive_index

    return beta

def calculate_beta_proton(energy_gev):
    # Rest mass energy of a proton in MeV
    rest_mass_energy_mev = 938.272

    # Convert energy from GeV to MeV
    energy_mev = energy_gev * 1000

    # Calculate the Lorentz factor
    gamma = 1 + (energy_mev / rest_mass_energy_mev)

    # Calculate beta using the Lorentz factor
    beta = np.sqrt(1 - 1 / gamma**2)

    return beta

def calculate_radius(ene, n=1.05, h=20, L=765):
    beta = calculate_beta_proton(ene)
    theta = np.arccos(1/(beta*1.05))

    return L * np.tan(np.arcsin(n * np.sin(theta))) + (h / 2) * np.tan(theta)

In [ ]:
import numpy as np
from scipy.optimize import curve_fit

# Define a Gaussian function for the fit
def gaussian(x, height, center, width):
    return height * np.exp(-(x - center)**2 / (2 * width**2))

def calculate_ratio(counts, bins, popt, specific_value):
    # Calculate the bin centers
    bin_centers = (bins[:-1] + bins[1:]) / 2

    # Calculate the mean and standard deviation from the Gaussian fit parameters
    mean = specific_value
    std_dev = popt[2]

    # Calculate the number of events within 2 sigmas of the specific value
    mask = (bin_centers > (mean - 2 * std_dev)) & (bin_centers < (mean + 2 * std_dev))
    events_within_2_sigmas = np.sum(counts[mask])

    # Calculate the total number of events
    total_events = np.sum(counts)

    # Calculate the ratio
    ratio = events_within_2_sigmas / total_events

    return ratio

# Drop duplicates based on 'eventNumber' and keep the first occurrence
df_good = df_good.drop_duplicates(subset='eventNumber', keep='first')

# Extract the data from the dataframe
beta_hat = df_good['beta_hat'].values

bin_width = 0.0001

# Calculate the number of bins for 'beta_hat'
bins_beta_hat = np.arange(min(beta_hat), max(beta_hat) + bin_width, bin_width)

# Create a histogram for 'beta_hat' and get the bin counts
counts_beta_hat, _ = np.histogram(beta_hat, bins=bins_beta_hat)

# Calculate the bin centers for 'beta_hat'
bin_centers_beta_hat = (bins_beta_hat[:-1] + bins_beta_hat[1:]) / 2

# Perform a Gaussian fit on 'beta_hat'
popt_beta_hat, _ = curve_fit(gaussian, bin_centers_beta_hat, counts_beta_hat, p0=[1, 0, 1])

# Calculate the ratio for 'beta_hat'
ratio = calculate_ratio(counts_beta_hat, bins_beta_hat, popt_beta_hat, calculate_beta_proton(ene))

# Store the ratios in a vector
ratios = np.array([ratio])

In [ ]:
ratios

array([0.00032867])

In [ ]:
0.00032867]